In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import ultraplot as uplt
from scipy import stats
from Py6S import *
import numpy.ma as ma
import os

from scipy.optimize import minimize

In [ ]:
SixS.test()

# Hotspot Identification

In [ ]:
dataset_path = os.path.join('datasets', 'palisades_fire')
pal_rad_path = os.path.join(dataset_path, 'AV320250111t210400_005_L1B_RDN_3f4aef90_RDN.nc')
pal_mask_path = os.path.join(dataset_path, 'AV320250111t210400_005_L1B_RDN_3f4aef90_BANDMASK.nc')
pal_ds = xr.open_datatree(pal_rad_path)
pal_ds

In [ ]:
pal_obs_path = os.path.join(dataset_path, 'AV320250111t210400_005_L1B_ORT_8827a51f_OBS.nc')
obs_ds = xr.open_datatree(pal_obs_path) # Observational parameters
obs_ds 

In [ ]:
samples_coords = np.arange(1234)
lines_coords = np.arange(1280)
# Assign dummy coordinates
pal_radiance = pal_ds.radiance.radiance.assign_coords({'samples':samples_coords, 'lines':lines_coords})
pal_radiance

In [ ]:
# observation parameters
obs_params = obs_ds.observation_parameters.to_dataset().assign_coords({'samples':samples_coords, 'lines':lines_coords})
obs_params

# Sample Images/Plots

In [ ]:
# Generate an RGB image
def normalize(band):
    band_min = band.min()
    band_max = band.max()
    return (band - band_min) / (band_max - band_min)

red_ = pal_radiance.sel(wavelength=700, method='nearest')
green_ = pal_radiance.sel(wavelength=550, method='nearest')
blue_ = pal_radiance.sel(wavelength=400, method='nearest')

red = normalize(red_)
green = normalize(green_) 
blue = normalize(blue_)
rgb_image = np.dstack((red.values, green.values, blue.values))
plt.figure(figsize=(5, 5))
plt.imshow(rgb_image)

In [ ]:
red_ = pal_radiance.sel(wavelength=2200, method='nearest')
green_ = pal_radiance.sel(wavelength=700, method='nearest')
blue_ = pal_radiance.sel(wavelength=550, method='nearest')

red = normalize(red_)
green = normalize(green_)
blue = normalize(blue_)
rgb_image = np.dstack((red.values, green.values, blue.values))
plt.figure(figsize=(5, 5))
plt.imshow(rgb_image)

In [ ]:
# Visualize the observation parameters
fig, ax = uplt.subplots(ncols=2)
obs_params.path_length.plot(ax=ax[0])
ax[0].format(
    title='Path Length',
    yreverse=True,
)

obs_params.cosine_i.plot(ax=ax[1])
ax[1].format(
    title='cosine_i',
    yreverse=True,
)

In [ ]:
# Visualize the observation parameters
fig, axs = uplt.subplots(nrows=2, ncols=2)

ax = axs[0,0]
obs_params.to_sun_zenith.plot(ax=ax)
ax.format(
    title='Solar Zenith Angle',
    yreverse=True,
)

ax = axs[0,1]
obs_params.to_sun_azimuth.plot(ax=ax)
ax.format(
    title='Solar Azimuth Angle',
    yreverse=True,
)

ax = axs[1,0]
obs_params.to_sensor_zenith.plot(ax=ax)
ax.format(
    title='Sensor Zenith Angle',
    yreverse=True,
)

ax = axs[1,1]
obs_params.to_sensor_azimuth.plot(ax=ax)
ax.format(
    title='Sensor Azimuth Angle',
    yreverse=True,
)

Both the solar zenith angle and azimuth angle can be treated as approximately constant throughout the read area.
As for the sensor zenith angle and azimuth angle, there is considerable variation in the angles across the sample dimension, because the sensor is read from a pushbroom sensor on an aircraft. Therefore from an aircraft altitude, the line-of-sight angle from left-to-right across the pushbroom varies significantly.

# Simple Atmospheric Correction

In [ ]:
import inspect

In [ ]:
# From path length we assume an altitude of 5km
# Create a 6S object from the viewing 
view = SixS()
view.altitudes.set_target_sea_level()
view.altitudes.set_sensor_custom_altitude(
    altitude=5 # 5km altitude
)
# Set atmospheric profiles; Data from Table 2-2 in http://www.exelisvis.com/docs/FLAASH.html
# For Los Angeles at a 34N latitude, recommended to set MidlatitudeSummer
view.atmos_profile = AtmosProfile.PredefinedType(AtmosProfile.MidlatitudeSummer)
# Aerosol profiles with pre-defined type
view.aero_profile = AeroProfile.PredefinedType(AeroProfile.BiomassBurning)
# Configure the sensor geometry
view.geometry = Geometry.User()
view.geometry.solar_z = 57.75
view.geometry.solar_a = 197.55
view.geometry.view_z = 0 # Assume fully Nadir-viewing
view.geometry.view_a = 0 # Consistent with Nadir-view

In [ ]:
# Set the wavelengths for the simulation
wavelengths = pal_radiance.wavelength.values/1000 # Wavelengths in micrometres
wv, res = SixSHelpers.Wavelengths.run_wavelengths(view, wavelengths)

In [ ]:
inspect.getmembers(res[0])

In [ ]:
# This produces an array of Py6S output objects arranged by wavelength
res_T_gas = np.array([s.total_gaseous_transmittance for s in res]) # Total gaseous transmittance
res_T_water = np.array([s.transmittance_water.total for s in res]) # Water vapour transmittance
res_T_up = np.array([s.transmittance_total_scattering.upward for s in res]) # Upward scattering transmittance
res_T_down = np.array([s.transmittance_total_scattering.downward for s in res]) # Downward scattering transmittance
# Get atmospheric intrinsic reflectance, which is scattering path radiance multiplied by transmittance
res_atm_ref = np.array([s.atmospheric_intrinsic_reflectance for s in res])


In [ ]:
np.argwhere(np.isnan(res_T_up))

In [ ]:
# Linearly interp the transmittance gaps
def interp_nans(array):
    xp = np.arange(len(array))
    # Get nans
    notnan = ~np.isnan(array)
    return np.interp(xp, xp[notnan], array[notnan])

res_T_up = interp_nans(res_T_up)
res_T_down = interp_nans(res_T_down)
res_atm_ref = interp_nans(res_atm_ref)

In [ ]:
fig, ax = uplt.subplots(refwidth=6, refaspect=(2,1))
ax.plot(wv, res_T_up, label='upward transmittance')
ax.plot(wv, res_T_down, label='downward transmittance')
ax.plot(wv, res_T_gas, label='gaseous transmittance')
ax.plot(wv, res_atm_ref, label='atmospheric intrinsic reflectance')
fig.legend(loc='b', ncols=2)


In [ ]:
# Choose a random pixel to do atmospheric correction as a test
pixel = pal_radiance.sel(lines=100, samples=1000)


fig, ax = uplt.subplots(refwidth=6, refaspect=(2,1))
ax.plot(wv, res_T_up, label='upward transmittance')
ax.plot(wv, res_T_down, label='downward transmittance')
ax.plot(wv, res_T_gas, label='gaseous transmittance')
ax.plot(wv, res_atm_ref, label='atmospheric intrinsic reflectance')
ax.plot(wv, pixel, label='pixel spectrum', c='k')
fig.legend(loc='b', ncols=2)


In [ ]:
# From the notes; relation of TOA radiance with ground radiance;
# L_TOA = (L_ground/(1-S rho) T_up T_down + L_path) Tg
# Neglecting the contribution from spherical albedo
# L_TOA = L_ground T_up T_down Tg + atm_intr_refl
# L_ground = (L_TOA - atm_intr_refl)/(T_up T_down T_g)

pixel_ground = (pixel - res_atm_ref) / (res_T_up * res_T_down * res_T_gas)

In [ ]:
fig, ax = uplt.subplots(refwidth=6, refaspect=(2,1))
ax.plot(wv, res_T_gas, label='Gas trasmittance')
ax.plot(wv, pixel, label='pixel spectrum', c='k')
ax.plot(wv, pixel_ground, label='corrected pixel spectrum', c='r')
ax.axhline(0.6, c='k', linestyle='--', lw=.5)
fig.legend(loc='b', ncols=2)
ax.format(
    ylim=(0,5)
)


In [ ]:
# We define problematic bands and mask them
# Mask all regions with less than 0.6 gaseous transmittance
# This is a tradeoff to prevent large losses due to absorption spikes
T_gas_cutoff = 0.6
wv_mask = res_T_gas < T_gas_cutoff

In [ ]:
fig, ax = uplt.subplots(refwidth=6, refaspect=(2,1))
ax.plot(wv, res_T_gas, label='Gas trasmittance')
ax.plot(wv, pixel, label='pixel spectrum', c='k')
ax.plot(wv, pixel_ground.where(~wv_mask), label='corrected pixel spectrum', c='r')
ax.axhline(0.6, c='k', linestyle='--', lw=.5)
fig.legend(loc='b', ncols=2)
ax.format(
    ylim=(0,5)
)

In [ ]:
pixel_ground.where(~wv_mask).sel(wavelength=2430, method='nearest')

## Dataset correction

In [ ]:
pal_radiance.coords['wavelength']

In [ ]:
# For xarray broadcasting, all of the terms need to be cast into dataarrays
res_atm_ref_da = xr.DataArray(
    res_atm_ref,
    dims=('wavelength'),
    coords={'wavelength': pal_radiance.coords['wavelength']}
)
res_T_up_da = xr.DataArray(
    res_T_up,
    dims=('wavelength'),
    coords={'wavelength': pal_radiance.coords['wavelength']}
)
res_T_down_da = xr.DataArray(
    res_T_down,
    dims=('wavelength'),
    coords={'wavelength': pal_radiance.coords['wavelength']}
)
res_T_gas_da = xr.DataArray(
    res_T_gas,
    dims=('wavelength'),
    coords={'wavelength': pal_radiance.coords['wavelength']}
)

In [ ]:
wv_mask = res_T_gas_da < T_gas_cutoff

In [ ]:
ground_radiance = (pal_radiance - res_atm_ref_da) / (res_T_up_da * res_T_down_da)

In [ ]:
red_ = ground_radiance.sel(wavelength=700, method='nearest')
green_ = ground_radiance.sel(wavelength=550, method='nearest')
blue_ = ground_radiance.sel(wavelength=400, method='nearest')

red = normalize(red_)
green = normalize(green_) 
blue = normalize(blue_)
rgb_image = np.dstack((red.values, green.values, blue.values))
plt.figure(figsize=(5, 5))
plt.imshow(rgb_image)

In [ ]:
red_ = ground_radiance.sel(wavelength=2200, method='nearest')
green_ = ground_radiance.sel(wavelength=700, method='nearest')
blue_ = ground_radiance.sel(wavelength=550, method='nearest')

red = normalize(red_)
green = normalize(green_)
blue = normalize(blue_)
rgb_image = np.dstack((red.values, green.values, blue.values))
plt.figure(figsize=(5, 5))
plt.imshow(rgb_image)

This simplistic atmospheric correction is very limited because it does not take into account:

- Topography (cos_i) angle
- Gaseous transmittance
- Aerosol scattering from burning plumes
- View angle and inhomogenous path lengths

# HFDI Hotspot

In [ ]:
# Calculate the HFDI index
# Remember that HFDI was designed to be robust against atmospheric absorption
pal_rad_2430 = pal_radiance.sel(wavelength=slice(2420,2440)).mean(dim='wavelength')
pal_rad_2060 = pal_radiance.sel(wavelength=slice(2050,2070)).mean(dim='wavelength')
pal_HFDI = (pal_rad_2430 - pal_rad_2060)/(pal_rad_2430 + pal_rad_2060)

In [ ]:
fig, ax = uplt.subplots(refwidth=6)
pal_HFDI.plot(ax=ax, vmin=-.5, vmax=.5, discrete=False, cmap='RdBu_r')
ax.format(
    yreverse=True,
    suptitle='Palisades Fire 2025-01-11 HFDI Index'
)

In [ ]:
bins = np.linspace(-0.4, 0.4, 500)
fig, ax = uplt.subplots(refwidth=6, refaspect=(3,1))
_ = pal_HFDI.plot.hist(bins=bins, ax=ax)
ax.format(
    suptitle='Distribution of pixel HFDI'
)

This distribution looks like a skew-normal with a long-tail anomaly. Assume that background pixels follow a skew-normal distribution, and use this to determine an appropriate threshold.

In [ ]:
# Define a fire mask
fire_mask = (pal_HFDI>0.1)
background_mask = (pal_HFDI<0.1)

In [ ]:
fig, ax = uplt.subplots(refwidth=6)
fire_mask.plot(ax=ax, vmin=-.5, vmax=.5, discrete=False, cmap='RdBu_r')
ax.plot(1120, 100, marker='o', c='r', s=1)
ax.format(
    yreverse=True,
    suptitle='Palisades Fire 2025-01-11 HFDI > 0.01'
)

In [ ]:
# Now find the region in shade
fig, ax = uplt.subplots()
(np.absolute(obs_params.cosine_i) < 0.05).plot(ax=ax)
ax.plot(800, 640, marker='o', c='r', s=1)

# Test with individual pixels

In [ ]:
# choose a fire pixel
fire_pixel = ground_radiance.sel(samples=1200, lines=100)
print('Fire pixel HFDI:', pal_HFDI.sel(samples=1120, lines=100).values)
print('Unburnt pixel cosi:', obs_params.cosine_i.sel(samples=1120, lines=100).values)
# choose an unburnt pixel
unburnt_pixel = ground_radiance.sel(samples=200, lines=600)
print('Unburnt pixel HFDI:', pal_HFDI.sel(samples=200, lines=600).values)
print('Unburnt pixel cosi:', obs_params.cosine_i.sel(samples=200, lines=600).values)
# choose a pixel in shade
shade_pixel = ground_radiance.sel(samples=800, lines=640)
print('Shade pixel HFDI:', pal_HFDI.sel(samples=800, lines=640).values)
print('Shade pixel cosi:', obs_params.cosine_i.sel(samples=800, lines=640).values)

In [ ]:
# Plot the test spectra
fig, ax = uplt.subplots(refwidth=5, refaspect=(2,1))
ax.plot(fire_pixel.where(~wv_mask), label='Fire pixel')
ax.plot(unburnt_pixel.where(~wv_mask), label='Unburnt pixel')
ax.plot(shade_pixel.where(~wv_mask), label='Shaded pixel')
ax.legend()

In [ ]:
# Should we restrict to a SWIR range?
fig, ax = uplt.subplots(refwidth=5, refaspect=(2,1))
ax.plot(fire_pixel - shade_pixel.where(~wv_mask), label='Fire pixel')
ax.plot(unburnt_pixel - shade_pixel.where(~wv_mask), label='Unburnt pixel')
ax.legend()

In [ ]:
## Define the linear mixing spectral model
## Nonlinear optimization is performed with respect to a simple L^2 loss
from scipy import constants as c
from scipy.interpolate import make_splrep

class SpectralMixtureModel():
    # Goal of the model is to infer
    # 1) n_fire of T_i (temperatures)
    # 2) n_fire of p_i (fire area fractions)
    # 3) n_bkg of p_j (fire area fractions)

    # There is a need to remove the reflected component of the land surface
    # Either from direct sunlight or indirect illumination by the sky

    def __init__(
        self,
        n_fire,
        n_bkg,
        bkg_spectra_lis, # List of spectra of len n_bkg
        # All spectra should be spectral radiances following lambd
    ):
        # Design this class to be able to hold arbitrary number of endmembers
        # But for actual purposes, have only one to two end members
        self.n_fire = n_bkg
        self.n_bkg = n_bkg
        
        if len(bkg_spectra_lis) != n_bkg:
            raise ValueError("n_bkg must be the same as the length of bkg_spectra_lis!")
        self.bkg_spectra_lis = bkg_spectra_lis

        # Create spline functions for the spectra
        bkg_spectra_splines = []
        for lambd, spectra in bkg_spectra_lis:
            spline = make_splrep(
                lambd,
                spectra,
                k=1,
                s=0
            )
            bkg_spectra_splines.append(spline)

        self.bkg_spectra_splines = bkg_spectra_splines

    def print_properties(self):
        print("Number of fire endmembers:", n_fire)
        print("Number of background endmembers:", n_bkg)
        print("Area fraction of fire endmembers:", T_fracs)
        print("Area fraction of background endmembers:", bkg_fracs)

    def _planck(self, T, lambd):
        top = 2 * c.h * c.c**2
        bottom = lambd**5 * (np.exp((c.h * c.c)/(c.k * T)) - 1)
        return top/bottom
    
    def get_fire_spectra(self, lambd, T_tup):
        result_list = list()
        for T in T_tup:
            spectra = self._planck(T, lambd)
            result_list.append(spectra)
        return np.array(result_list)
    
    def get_bkg_spectra(self, lambd):
        result_list = list()
        for spline in self.bkg_spectra_splines:
            result_list.append(spline(lambd))
        return np.array(result_list)

    def total_radiance(self, lambd, T_tup, T_fracs, bkg_fracs):
        # noise is given as an absolute radiance value for 1 std
        fire_spectra = self.get_fire_spectra(lambd, T_tup)
        bkg_spectra = self.get_bkg_spectra(lambd)

        result = np.zeros_like(lambd)
        for frac, spectra in zip(T_fracs, fire_spectra):
            result += frac * spectra

        for frac, spectra in zip(bkg_fracs, bkg_spectra):
            result += frac * spectra
    
        return result

In [ ]:
bkg_spectra_1 = unburnt_pixel.where(~wv_mask).dropna(dim='wavelength')
bkg_spectra_1

In [ ]:
# Create a spectral mixture model instance
# Retrieve the unpurnt pixel spectra as the bkg spectra
mask_lambds = bkg_spectra_1.wavelength.values
simple_model = SpectralMixtureModel(
    n_fire=1,
    n_bkg=1,
    bkg_spectra_lis=[(mask_lambds, bkg_spectra_1.values)],    
)

In [ ]:
srad = simple_model.total_radiance(
    lambd=mask_lambds,
    T_tup = (1200,),
    T_fracs = (0.1,),
    bkg_fracs = (0.9,),
)

In [ ]:
# Plot the test spectra
fig, ax = uplt.subplots(refwidth=5, refaspect=(2,1))
ax.plot(fire_pixel.where(~wv_mask), label='Fire pixel')
ax.plot(unburnt_pixel.where(~wv_mask), label='Unburnt pixel')
ax.plot(mask_lambds, srad, label='mixture model ouput')
ax.legend()

In [ ]:
def retrieve_params(result, model):
    n_fire = model.n_fire
    n_bkg = model.n_bkg
    params = result.x
    T_tup = params[:n_fire] * 1000
    T_frac = params[n_fire:2*n_fire]
    if len(params) >= 2*n_fire:
        bkg_fracs = tuple(params[2*n_fire:]) + (1 - np.sum(params[n_fire:]),)
    else:
        bkg_fracs = (1 - np.sum(params[n_fire:]),)
    print("The fire temperatures in K are: ", T_tup)
    print("The fire fractions are: ", T_frac)
    print("The background fractions are: ", bkg_fracs)
    return T_tup, T_frac, bkg_fracs

# T, T_frac, bkg_frac = retrieve_params(srad)

## Applying the Spectral Mixture Model (2 fire endmembers and 1 background endmember)

In [ ]:
simple_model = SpectralMixtureModel(
    n_fire=2,
    n_bkg=1,
    bkg_spectra_lis=[(mask_lambds, bkg_spectra_1.values)]
)

In [ ]:
# Model inversion
# Define the loss function

def return_loss(model, lambd, target):
    n_fire = model.n_fire
    n_bkg = model.n_bkg

    # The parameter vector is an ndarray of shape (2*n_fire + n_bkg-1,)
    # Arguments are organized in the following order
    # [T_i, p_i_fire, p*_j_bkg]
    # Note that in order to satisfy the p_i_fire + p_j_bkg = 1 constraint,
    # The last p_j_bkg parameter is omitted and calculated from 1 - all

    def loss(params):
        # Unpact the parameters and normalize them appropriately
        T_tup = tuple(params[:n_fire]*1000) # 1000 K scale
        T_fracs = tuple(params[n_fire:2*n_fire])
        # Enforcing land fraction constraint
        if len(params) >= 2*n_fire:
            bkg_fracs = tuple(params[2*n_fire:]) + (1 - np.sum(params[n_fire:]),)
        else:
            bkg_fracs = (1 - np.sum(params[n_fire:]),)

        prediction = model.total_radiance(lambd, T_tup, T_fracs, bkg_fracs)
        diff = target - prediction
        return np.sum(diff**2) # L2 norm

    return loss

In [ ]:
def planck_radiance(wavelength_nm, T, emissivity=1.0):
    """Returns spectral radiance [W/m^2/sr/nm] at a given temperature."""
    wavelength_m = wavelength_nm * 1e-9
    term1 = 2 * c.h * c.c**2 / wavelength_m**5
    exponent = c.h * c.c / (wavelength_m * c.k * T)
    term2 = 1 / (np.exp(exponent) - 1)
    return emissivity * term1 * term2 * 1e-9  # Convert to per nm

def model_radiance(params, wavelengths, L_bkg, L_obs):
    T1, T2, p1, p2 = params
    if not (0 <= p1 <= 1 and 0 <= p2 <= 1 and p1 + p2 <= 1):
        return 1e6  # penalize invalid fractions
    p_b = 1 - p1 - p2
    L1 = planck_radiance(wavelengths, T1)
    L2 = planck_radiance(wavelengths, T2)
    L_model = p1 * L1 + p2 * L2 + p_b * L_bkg
    return np.sqrt(np.mean((L_model - L_obs)**2))  # RMSE

In [ ]:
wavelengths = wavelengths * 1e3 # Converting to nm
mask = (mask_lambds > 1000)  # SWIR window
usable_lambds = mask_lambds[mask]

In [ ]:
x = pal_radiance.sel(lines=98, samples=148)
x.sel(wavelength=usable_lambds, method='nearest')

In [ ]:
# Compiling the coordinates of all background points (negative HFDI)
background_lines = []
background_samples = []

line = 0
for row in background_mask:
    sample = 0
    for boolean in row:
        if boolean == True:
            background_samples.append(sample)
            background_lines.append(line)
        sample += 1
    line += 1

background_coordinates = [(background_lines[i], background_samples[i]) for i in range(len(background_lines))]

In [ ]:
fire_pixel = pal_radiance.sel(lines=100, samples=1100, wavelength=usable_lambds, method='nearest')
fire_pixel_ = pal_radiance.sel(lines= 100, samples=1100, method='nearest')
fire_pixel_ = fire_pixel_.where(~wv_mask)


pixel_HFDI = pal_HFDI.sel(lines=100, samples=1100)

# # Select 15 random background pixels
lst_of_background_pixels = []
for i in range(15):
    random_index = np.random.randint(0, len(background_coordinates))
    lst_of_background_pixels.append(pal_radiance.sel(lines=background_coordinates[random_index][0],
                                                     samples=background_coordinates[random_index][1],
                                                     wavelength=usable_lambds,
                                                     method='nearest'))


bkg_pixels = np.array(lst_of_background_pixels)
bkg_pixels

In [ ]:
# Perform fitting N times
import random

def monte_carlo_real_data(L_obs_clean, bkg_pool, wavelengths, n_runs=15, noise_std=0.01):
    results = []

    for _ in range(n_runs):
        # Pick one random background spectrum
        L_obs_clean
        L_b = bkg_pool[random.randint(0, len(bkg_pixels) - 1)]
        
        # Add noise
        L_obs_noisy = L_obs_clean + np.random.normal(0, noise_std, size=L_obs_clean.shape)

        # Fit
        x0 = [1200, 600, 
            #   700, 600, # for showing optimisation scheme falling into local minima and giving bad results for temp estimation
            #   820, 600  # for showing optimisation scheme falling into local minima and giving bad results for temp estimation
              0.1, 0.1]
        bounds = [(600, 1500), (400, 1000), (0, 1), (0, 1)]
        result = minimize(
            model_radiance,
            x0,
            args=(wavelengths, L_b, L_obs_noisy),
            bounds=bounds,
            method='L-BFGS-B'
        )

        if result.success:
            results.append(result.x)

    results = np.array(results)
    if len(results) == 0:
        return None

    means = np.mean(results, axis=0)
    stds = np.std(results, axis=0)

    return {
        "T1_mean": means[0], "T1_std": stds[0],
        "T2_mean": means[1], "T2_std": stds[1],
        "p1_mean": means[2], "p1_std": stds[2],
        "p2_mean": means[3], "p2_std": stds[3],
        "p_bkg_mean": 1 - means[2] - means[3],
        "p_bkg_std": np.sqrt(stds[2]**2 + stds[3]**2)
    }

# Run it
result = monte_carlo_real_data(fire_pixel, bkg_pixels, usable_lambds)

if result:
    print(result)

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))

T1_mean = result['T1_mean']
T2_mean = result['T2_mean']
p1_mean = result['p1_mean']
p2_mean = result['p2_mean']


fire1_contribution = planck_radiance(usable_lambds, result['T1_mean'])
fire2_contribution = planck_radiance(usable_lambds, result['T2_mean'])

lmm_output = result['p1_mean'] * fire1_contribution + result['p2_mean'] * fire2_contribution + result['p_bkg_mean'] * np.mean(bkg_pixels, axis=0)

# ax.plot(usable_wavelengths, result['p1_mean'] * fire1_contribution, label='fire1')
# ax.plot(usable_wavelengths, result['p2_mean'] * fire2_contribution, label='fire2')
# ax.plot(usable_wavelengths, result['p_bkg_mean'] * np.mean(bkg_pixels, axis=0), label='background')
# ax.set_title('2 fire contribution terms and 1 background contribution term')
ax.plot(wavelengths/1e3, fire_pixel_, label='Fire pixel')
ax.plot(usable_lambds, lmm_output, label=rf'Fitted spectra, $T_1 =$ {T1_mean:.2f}K, $p_1 =$ {p1_mean:.5f}, $T_2 =$ {T2_mean:.2f}K, $p_2 =$ {p2_mean:.5f}')

ax.legend()

ax.set_xlabel('Wavelength centers (nm)')
ax.set_ylabel(r'Spectral radiance (W/$\text{m}^2$ sr nm)')
ax.set_title(f'Fit_results, n_bkg=1, n_fire=2, pixel_HFDI={pixel_HFDI:.2f}')

fig_path = os.path.join('figs', 'aviris_plots')
plt.savefig(os.path.join(fig_path, 'aviris_bad(localminima).png'), dpi=300)